In [5]:
import project_path

import pandas as pd
from sof_sa.utility_functions import connect2db, db_config

creds = db_config("..\\dev_dbconfig.yaml", 0)
engine = connect2db(**creds)

In [8]:
from sqlalchemy import text

sql = text('select * from pl_salary_staging')
result = engine.execute(sql)
result.fetchall()

[(0, 'Perl', '$76,000'),
 (1, 'Scala', '$76,000'),
 (2, 'Go', '$74,000'),
 (3, 'Rust', '$74,000'),
 (4, 'Ruby', '$71,000'),
 (5, 'Bash/Shell/PowerShell', '$65,000'),
 (6, 'Objective-C', '$64,000'),
 (7, 'Haskell', '$60,000'),
 (8, 'Julia', '$59,000'),
 (9, 'Python', '$59,000'),
 (10, 'Swift', '$58,000'),
 (11, 'C#', '$57,000'),
 (12, 'R', '$57,000'),
 (13, 'TypeScript', '$57,000'),
 (14, 'Kotlin', '$54,000'),
 (15, 'SQL', '$54,000'),
 (16, 'Assembly', '$53,000'),
 (17, 'C++', '$53,000'),
 (18, 'JavaScript', '$53,000'),
 (19, 'HTML/CSS', '$52,000'),
 (20, 'VBA', '$51,000'),
 (21, 'C', '$50,000'),
 (22, 'Java', '$50,000'),
 (23, 'PHP', '$39,000'),
 (24, 'Dart', '$37,000')]